In [6]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE

# Загрузка данных

In [ ]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [17]:
data = pd.read_excel('data/data_ford_price.xlsx') 

In [18]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [19]:
# определяем 3 лучших признака методом рекурсивного исключения признаков
estimator = LinearRegression()
selector1 = RFE(estimator, n_features_to_select=3, step=1)
selector1 = selector1.fit(X_train, y_train)
 
selector1.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [23]:
# оставляем целевой и отобранные признаки 
data1 = data[['price','year', 'cylinders', 'lat']]
data1.dropna(inplace = True)

y = data1['price']
x = data1.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
# обучаем модель 
model = LinearRegression()
model.fit(X_train, y_train)
# делаем предсказание на тесте
y_predicted = model.predict(X_test)
# определеяем метрику средней абсолютной ошибки 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE для метода RFE: %.3f' % mae)

MAE для метода RFE: 5096.570


/var/folders/jr/y6j710014w98cjhsh00zh_gw0000gn/T/ipykernel_6286/665955094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.dropna(inplace = True)


In [26]:
# определяем 3 лучших признака методом SelectKBest
selector2 = SelectKBest(f_regression, k=3)
selector2.fit(X_train, y_train)
 
selector2.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [27]:
# оставляем целевой и отобранные признаки 
data2 = data[['price','year', 'cylinders', 'odometer']]
data2.dropna(inplace = True)

y = data2['price']
x = data2.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)
# обучаем модель 
model = LinearRegression()
model.fit(X_train, y_train)
# делаем предсказание на тесте
y_predicted = model.predict(X_test)
# определеяем метрику средней абсолютной ошибки  
mae = mean_absolute_error(y_test, y_predicted)
print('MAE для метода SelectKBest: %.3f' % mae)

MAE для метода SelectKBest: 4708.946


/var/folders/jr/y6j710014w98cjhsh00zh_gw0000gn/T/ipykernel_6286/1204168186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.dropna(inplace = True)


**Вывод: \
для данного датасета лучшим вариантом будет использование метода SelectKBest по отбору наилучших признаков для модели.\
На тестовых данных метрика MAE для данного метода составила 4709 против 5097 - значение нашей метрики для метода RFE**